# Package

In [58]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
from sklearn.utils import shuffle
import pickle

# Importation

In [59]:
# Les données
df_stationary_test = pd.read_csv("df_stationary_test.csv", index_col="date")
df_stationary_test.index = pd.to_datetime(df_stationary_test.index)

In [60]:
df_stationary_train = pd.read_csv("df_stationary_train.csv", index_col="date")
df_stationary_train.index = pd.to_datetime(df_stationary_train.index)

# 1) Paramètres & sous-ensemble TRAIN global

In [61]:
from dateutil.relativedelta import relativedelta

In [62]:
# ---------- Paramètres ----------
window_len = 12      # longueur minimale de la fenêtre (non utilisé si expanding)
step_size = 12       # refit annuel (adapter : 1 = mensuel, 12 = annuel)
winsor_level = 0.01
# === Bloc 1bis — Paramètre de décalage (12 mois) ===
SHIFT_LAG = 12  # décalage des features : X_{t-12} -> y_t
h = 12

In [63]:
# TRAIN global
df_train_global = df_stationary_train.sort_index().copy()

# ✅ Inclure toutes les features (aucune exclusion sauf la cible)
features = [c for c in df_stationary_train.columns if c != "UNRATE"]
cols_tx = features.copy()

# ---------- Bornes de validation ----------
valid_start = pd.Timestamp("1983-01-01")
valid_end   = pd.Timestamp("1989-12-31")

In [64]:
# === Bloc 3bis — Pré-calcul des features décalées ===
# (Optionnel mais plus propre/rapide que de refaire .shift() à chaque itération)
df_tx_shifted = df_train_global[cols_tx].shift(SHIFT_LAG)

In [65]:
# === Bloc 2 — Helpers de prétraitement (USREC exemptée), + métriques ===

def fit_preproc(X, winsor_level=0.01, do_norm=True, exempt_cols=None):
    """
    Apprend les bornes de winsorisation et les stats de normalisation sur X,
    mais n'applique rien aux colonnes 'exempt_cols' (ex: ['USREC']).
    """
    exempt_cols = list(exempt_cols or [])
    cols_w = [c for c in X.columns if c not in exempt_cols]

    Xw = X.copy()

    # Winsorisation uniquement sur les colonnes non-exemptées
    if len(cols_w):
        lower_wins = X[cols_w].quantile(winsor_level)
        upper_wins = X[cols_w].quantile(1 - winsor_level)
        Xw[cols_w] = Xw[cols_w].clip(lower=lower_wins, upper=upper_wins, axis=1)
    else:
        lower_wins = pd.Series(dtype=float)
        upper_wins = pd.Series(dtype=float)

    # Normalisation uniquement sur les colonnes non-exemptées
    if do_norm and len(cols_w):
        mean = Xw[cols_w].mean()
        std  = Xw[cols_w].std().replace(0, 1)
        Xn = Xw.copy()
        Xn[cols_w] = (Xw[cols_w] - mean) / std
    else:
        mean, std = None, None
        Xn = Xw

    return Xn, {
        "lower_wins": lower_wins,
        "upper_wins": upper_wins,
        "mean": mean,
        "std": std,
        "norm": do_norm,
        "exempt_cols": exempt_cols
    }


def apply_preproc(X, prep):
    """
    Applique les bornes/stats apprises sur les colonnes non-exemptées.
    Les colonnes exemptées (ex: USREC) restent brutes.
    """
    exempt_cols = list(prep.get("exempt_cols", []))
    cols_w = [c for c in X.columns if c not in exempt_cols]

    Xp = X.copy()
    if len(cols_w):
        # Appliquer les mêmes bornes/paramètres sur les colonnes non-exemptées
        lw = prep["lower_wins"].reindex(cols_w)
        uw = prep["upper_wins"].reindex(cols_w)
        Xp[cols_w] = Xp[cols_w].clip(lower=lw, upper=uw, axis=1)

        if prep["norm"]:
            mean = prep["mean"].reindex(cols_w)
            std  = prep["std"].reindex(cols_w).replace(0, 1)
            Xp[cols_w] = (Xp[cols_w] - mean) / std

    # Les colonnes exemptées (ex. USREC) restent brutes
    return Xp


def perf_report(y_true, y_pred):
    # RMSE = racine du MSE manuellement pour compatibilité avec toutes les versions
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    return {"RMSE": float(rmse), "MAE": float(mae), "R2": float(r2)}

In [66]:
# === Bloc 3bis — Pré-calcul des features décalées ===
# (Optionnel mais plus propre/rapide que de refaire .shift() à chaque itération)
df_tx_shifted = df_train_global[cols_tx].shift(SHIFT_LAG)

In [67]:
# === Bloc 3 — Boucle d'entraînement (expanding) avec prédicteurs décalés de 12 mois ===

# ---------- Containers ----------
models = []
coefs = []
preprocs = []
train_periods = []
preds = []
norm_var = True  # True = normalisation des colonnes non-exemptées (USREC reste brute)

# ---------- Boucle EXPANDING WINDOW SUR LE TRAIN ----------
for t, end in enumerate(range(step_size, len(df_train_global), step_size)):
    df_train_local = df_train_global.iloc[:end].copy()

    # 0) Construire X_train décalé (X_{t-12}) et aligner y_t
    X_train_raw = df_tx_shifted.loc[df_train_local.index].copy()   # prédicteurs décalés
    y_train_full = df_train_local["UNRATE"].copy()

    # supprimer les premières lignes qui deviennent NaN à cause du décalage
    mask_valid = ~X_train_raw.isna().any(axis=1)
    X_train_raw = X_train_raw.loc[mask_valid]
    y_train = y_train_full.loc[X_train_raw.index]

    # guardrail : si pas assez d'observations, on saute cette itération
    if len(X_train_raw) < 2:
        print(f"[{t:02d}] Skip (pas assez d'obs après décalage).")
        continue

    # 1️⃣ Winsorisation/normalisation apprises sur TRAIN — USREC exclue
    X_train, prep = fit_preproc(
        X_train_raw,
        winsor_level=winsor_level,
        do_norm=norm_var,
        exempt_cols=["USREC"]  # ⬅️ USREC pas de winsor ni normalisation
    )

    # 2️⃣ Entraînement OLS
    model = LinearRegression()
    model.fit(X_train, y_train)

    # 3️⃣ Fenêtre de prédiction (année suivante) + intersection 1983–1989
    start_pred = (df_train_local.index[-1] + relativedelta(months=1)).replace(day=1)
    end_pred   = start_pred + relativedelta(months=12, days=-1)

    start_use = max(start_pred, valid_start)
    end_use   = min(end_pred, valid_end)

    if start_use <= end_use:
        # X_valid doit aussi être décalé de 12 mois
        X_valid_raw = df_tx_shifted.loc[start_use:end_use].copy()
        # enlever les éventuels NaN résiduels
        X_valid_raw = X_valid_raw.dropna(how="any")

        if not X_valid_raw.empty:
            # y_valid = y_t aligné au même index
            y_valid = df_train_global.loc[X_valid_raw.index, "UNRATE"].values

            # appliquer le prétraitement appris sur TRAIN
            X_valid = apply_preproc(X_valid_raw, prep)

            # prédire
            yhat = model.predict(X_valid)

            # stocker
            df_out = pd.DataFrame({"y_true": y_valid, "y_pred": yhat}, index=X_valid_raw.index)
            df_out["model_trained_until"] = df_train_local.index[-1]
            preds.append(df_out)

    # 4️⃣ Sauvegarde des éléments d'entraînement
    models.append(model)
    coefs.append(pd.Series(model.coef_, index=cols_tx))
    preprocs.append(prep)
    train_periods.append(df_train_local.index[-1])

    print(f"[{t:02d}] Fin {df_train_local.index[-1].strftime('%Y-%m')} — "
          f"shift={SHIFT_LAG}m — winsor({winsor_level:.2%}) + "
          f"{'norm' if norm_var else 'no-norm'} — {len(X_train)} obs.")

[00] Skip (pas assez d'obs après décalage).
[01] Fin 1961-12 — shift=12m — winsor(1.00%) + norm — 12 obs.
[02] Fin 1962-12 — shift=12m — winsor(1.00%) + norm — 24 obs.
[03] Fin 1963-12 — shift=12m — winsor(1.00%) + norm — 36 obs.
[04] Fin 1964-12 — shift=12m — winsor(1.00%) + norm — 48 obs.
[05] Fin 1965-12 — shift=12m — winsor(1.00%) + norm — 60 obs.
[06] Fin 1966-12 — shift=12m — winsor(1.00%) + norm — 72 obs.
[07] Fin 1967-12 — shift=12m — winsor(1.00%) + norm — 84 obs.
[08] Fin 1968-12 — shift=12m — winsor(1.00%) + norm — 96 obs.
[09] Fin 1969-12 — shift=12m — winsor(1.00%) + norm — 108 obs.
[10] Fin 1970-12 — shift=12m — winsor(1.00%) + norm — 120 obs.
[11] Fin 1971-12 — shift=12m — winsor(1.00%) + norm — 132 obs.
[12] Fin 1972-12 — shift=12m — winsor(1.00%) + norm — 144 obs.
[13] Fin 1973-12 — shift=12m — winsor(1.00%) + norm — 156 obs.
[14] Fin 1974-12 — shift=12m — winsor(1.00%) + norm — 168 obs.
[15] Fin 1975-12 — shift=12m — winsor(1.00%) + norm — 180 obs.
[16] Fin 1976-12 — 

In [68]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [69]:
# === Bloc 4 — (inchangé) Agrégation OOS + métriques + sauvegarde d'expérience ===
# (utilise 'preds' rempli ci-dessus ; USREC reste exclue du prétraitement)
df_oos = (pd.concat(preds).sort_index() if len(preds) else
          pd.DataFrame(columns=["y_true","y_pred"]))

m83 = (df_oos.index >= "1983-01-01") & (df_oos.index <= "1983-12-31")
perf_1983 = perf_report(df_oos.loc[m83, "y_true"], df_oos.loc[m83, "y_pred"]) if m83.any() else None

m8389 = (df_oos.index >= valid_start) & (df_oos.index <= valid_end)
perf_83_89 = perf_report(df_oos.loc[m8389, "y_true"], df_oos.loc[m8389, "y_pred"]) if m8389.any() else None

if not df_oos.empty:
    df_oos["year"] = df_oos.index.year
    annual_perf = (df_oos.loc[m8389]
                   .groupby("year")
                   .apply(lambda g: pd.Series(perf_report(g["y_true"], g["y_pred"])))
                   .to_dict(orient="index"))
else:
    annual_perf = {}

exp_results = {
    "models": models,
    "coefs": coefs,
    "train_periods": train_periods,
    "features": cols_tx,
    "preprocs": preprocs,
    "oos_predictions": df_oos,
    "annual_perf": annual_perf,
    "perf_1983": perf_1983,
    "perf_83_89": perf_83_89,
    "params": {
        "step_size": step_size,
        "winsor_level": winsor_level,
        "norm_var": norm_var,
        "valid_window": ("1983-01", "1989-12"),
        "window_len": window_len,
        "exempt_cols": ["USREC"],
        "shift_lag": SHIFT_LAG
    }
}

print("\n✅ Entraînement + validation 1983–1989 terminés (prédicteurs décalés de 12 mois).")


✅ Entraînement + validation 1983–1989 terminés (prédicteurs décalés de 12 mois).


C:\Users\Mita\AppData\Local\Temp\ipykernel_19192\3596606838.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series(perf_report(g["y_true"], g["y_pred"])))


In [70]:
import pandas as pd
import joblib

# Sauvegarde du dictionnaire de résultats
joblib.dump(exp_results, "linear_regression.pkl")
print("✅ Fichier 'linear_regression.pkl' sauvegardé.")

# =========================
# 🔹 Sauvegarde des métadonnées du modèle linéaire
# =========================

# On extrait les infos clés pour documenter le modèle
meta_info = {
    "trained_until": str(exp_results["train_periods"][-1]) if exp_results.get("train_periods") else None,
    "n_models": len(exp_results.get("models", [])),
    "features": len(exp_results.get("features", [])),
    "shift_lag": exp_results["params"].get("shift_lag"),
    "winsor_level": exp_results["params"].get("winsor_level"),
    "norm_var": exp_results["params"].get("norm_var"),
    "valid_window_start": exp_results["params"]["valid_window"][0],
    "valid_window_end": exp_results["params"]["valid_window"][1],
    "mae_83_89": exp_results["perf_83_89"]["MAE"] if exp_results.get("perf_83_89") else None,
    "rmse_83_89": exp_results["perf_83_89"]["RMSE"] if exp_results.get("perf_83_89") else None,
    "r2_83_89": exp_results["perf_83_89"]["R2"] if exp_results.get("perf_83_89") else None,
}

# Conversion en DataFrame pour export
meta_df = pd.DataFrame.from_dict(meta_info, orient="index", columns=["value"])

# Sauvegarde au format CSV
meta_df.to_csv("linear_regression_meta.csv")
print("✅ Fichier 'linear_regression_meta.csv' sauvegardé avec les métadonnées du modèle linéaire.")

✅ Fichier 'linear_regression.pkl' sauvegardé.
✅ Fichier 'linear_regression_meta.csv' sauvegardé avec les métadonnées du modèle linéaire.


In [38]:
# === Bloc 5 — Évaluation pseudo–OOS détaillée (1983–1989) ===
# (USREC est déjà traitée correctement via les blocs précédents)

from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 0️⃣ Vérifs de base
if "df_oos" not in globals():
    raise RuntimeError("df_oos est introuvable. Assure-toi d'avoir exécuté le bloc d'entraînement/prédiction.")

# 1️⃣ Construire forecast_df à partir de df_oos, restreint à 1983–1989
forecast_slice = df_oos.loc["1983-01-01":"1989-12-31"].copy()

# On crée explicitement 'target' depuis l'index (quel que soit son nom), et on renomme y_pred -> y_hat
forecast_df = (
    forecast_slice.rename(columns={"y_pred": "y_hat"})[["y_true", "y_hat"]]
                  .assign(target=forecast_slice.index)
                  .dropna(subset=["y_true", "y_hat"])
)
forecast_df["target"] = pd.to_datetime(forecast_df["target"])
forecast_df = forecast_df.sort_values("target").reset_index(drop=True)

# 2️⃣ Fonctions auxiliaires
def r2_origin_reg(y, yhat):
    """R² d'une régression à l’origine: y ≈ b * yhat (sans intercept)."""
    y = np.asarray(y); yhat = np.asarray(yhat)
    denom = np.dot(yhat, yhat)
    if denom == 0:
        return np.nan
    b = np.dot(yhat, y) / denom
    sse = np.sum((y - b * yhat) ** 2)
    sst = np.sum((y - y.mean()) ** 2)
    return 1 - sse / sst if sst > 0 else np.nan

def corr_pvalue(y, yhat):
    y = np.asarray(y); yhat = np.asarray(yhat)
    if len(y) < 3:
        return np.nan, np.nan
    r, p = pearsonr(y, yhat)
    return float(r), float(p)

# 3️⃣ Évaluation
if forecast_df.empty:
    print("\n[ÉVALUATION 83–89] Aucune prévision disponible (forecast_df est vide).")
    eval_results_83_89 = None
else:
    y    = forecast_df["y_true"].values.astype(float)
    yhat = forecast_df["y_hat"].values.astype(float)

    # Métriques principales
    r2   = r2_score(y, yhat)
    mae  = mean_absolute_error(y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))   # ✅ compatible toutes versions
    r2_o = r2_origin_reg(y, yhat)
    corr, pval = corr_pvalue(y, yhat)
    amd  = float(abs(np.mean(y - yhat)))          # biais absolu moyen

    # Benchmark naïf (zéro-changement)
    yhat_naive0 = np.zeros_like(y)
    mae_naive0  = mean_absolute_error(y, yhat_naive0)
    rmse_naive0 = np.sqrt(mean_squared_error(y, yhat_naive0))

    # Précision directionnelle
    hit_rate = float(np.mean(np.sign(y) == np.sign(yhat))) if len(y) > 0 else np.nan

    # Calibration (Mincer–Zarnowitz): y = a + b * yhat
    reg = LinearRegression()
    reg.fit(yhat.reshape(-1, 1), y)
    a_calib = float(reg.intercept_)
    b_calib = float(reg.coef_[0])

    # Résumé par année 1983…1989
    by_year = (
        forecast_df.assign(year=forecast_df["target"].dt.year)
                   .groupby("year")
                   .apply(lambda g: pd.Series({
                       "n": len(g),
                       "MAE": mean_absolute_error(g["y_true"], g["y_hat"]),
                       "RMSE": np.sqrt(mean_squared_error(g["y_true"], g["y_hat"]))
                   }))
                   .reset_index()
    )

    # Impression des résultats
    print("\n=== ÉVALUATION PSEUDO–OOS (1983–1989) ===")
    print(f"R²            : {r2:.3f}")
    print(f"R² (origin)   : {r2_o:.3f}")
    print(f"MAE           : {mae:.3f}")
    print(f"RMSE          : {rmse:.3f}")
    print(f"Corr(y, ŷ)   : {corr:.3f}  (p={pval:.3g})")
    print(f"Hit rate sign : {hit_rate:.3f}")
    print(f"AMD (|bias|)  : {amd:.3f}")

    print("\n--- Benchmark naïf (zéro-changement) ---")
    print(f"MAE_naïf0     : {mae_naive0:.3f}")
    print(f"RMSE_naïf0    : {rmse_naive0:.3f}")

    if not by_year.empty:
        print("\n--- MAE/RMSE par année (1983–1989) ---")
        print(by_year.to_string(index=False))

    # Sauvegarde des résultats
    eval_results_83_89 = {
        "overall": {
            "r2": float(r2), "r2_origin": float(r2_o), "mae": float(mae), "rmse": float(rmse),
            "corr": float(corr), "pval": float(pval), "hit_rate": float(hit_rate), "amd": float(amd)
        },
        "benchmark_naive0": {
            "mae": float(mae_naive0), "rmse": float(rmse_naive0)
        },
        "calibration": {
            "intercept": a_calib, "slope": b_calib
        },
        "by_year": by_year,
        "forecast_df": forecast_df
    }
    
    print("\n✅ Évaluation 1983–1989 terminée. Résultats dans eval_results_83_89.")


=== ÉVALUATION PSEUDO–OOS (1983–1989) ===
R²            : -0.300
R² (origin)   : -0.243
MAE           : 0.737
RMSE          : 1.009
Corr(y, ŷ)   : 0.424  (p=6.53e-05)
Hit rate sign : 0.711
AMD (|bias|)  : 0.534

--- Benchmark naïf (zéro-changement) ---
MAE_naïf0     : 0.813
RMSE_naïf0    : 1.095

--- MAE/RMSE par année (1983–1989) ---
 year    n      MAE     RMSE
 1983 12.0 1.213098 1.425113
 1984 12.0 1.760668 1.865827
 1985 12.0 0.437982 0.546664
 1986 12.0 0.310524 0.364724
 1987 12.0 0.686419 0.763033
 1988 12.0 0.544958 0.691979
 1989 11.0 0.157393 0.206414

✅ Évaluation 1983–1989 terminée. Résultats dans eval_results_83_89.


C:\Users\Mita\AppData\Local\Temp\ipykernel_19192\1257724684.py:77: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


Le modèle linéaire à 12 mois d’horizon prévoit la direction du chômage mieux qu’un modèle naïf, mais échoue à reproduire l’ampleur des variations, surtout lors des retournements conjoncturels.

In [39]:
# ==========================================================
# 🔍 IMPORTANCE PAR PERMUTATION — PSEUDO-OOS
# ==========================================================
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

def permutation_importance_pseudo_oos(models, df_train_global, cols_tx, h=12, n_repeats=20, metric=mean_absolute_error):
    """
    Importance par permutation pour une série de modèles OLS entraînés
    selon une logique expanding window pseudo–OOS.
    -> aucune réestimation
    -> mesure la dégradation moyenne de la performance après permutation de chaque variable
    """
    var_imp = {col: [] for col in cols_tx}

    for i, model in enumerate(models):
        # Reconstituer la fenêtre utilisée par le modèle i
        end_idx = (i + 1) * 12  # correspond à ton step_size = 12
        df_win = df_train_global.iloc[:end_idx].copy()

        if len(df_win) <= h:
            continue

        # Préparation des données (alignement X_t avec Y_{t+h})
        X = df_win[cols_tx].iloc[:-h].copy()
        y = df_win["UNRATE"].shift(-h).iloc[:-h].copy()
        valid = ~(X.isnull().any(axis=1) | y.isnull())
        X, y = X.loc[valid], y.loc[valid]

        # Score de base (MAE sur données d'origine)
        base_score = metric(y, model.predict(X))

        # Boucle sur chaque variable
        for col in cols_tx:
            perm_scores = []
            for _ in range(n_repeats):
                X_perm = X.copy()
                X_perm[col] = np.random.permutation(X_perm[col])
                perm_scores.append(metric(y, model.predict(X_perm)))
            perm_scores = np.array(perm_scores)
            var_imp[col].append(np.mean(perm_scores) / base_score)

    # Agrégation moyenne sur toutes les fenêtres
    results = []
    for col, ratios in var_imp.items():
        if len(ratios) > 0:
            results.append({
                "variable": col,
                "perm_mae_ratio_mean": np.mean(ratios),
                "perm_mae_ratio_std": np.std(ratios),
                "n_windows": len(ratios)
            })

    imp_df = pd.DataFrame(results).sort_values("perm_mae_ratio_mean", ascending=False).reset_index(drop=True)
    return imp_df

In [42]:
# ----------------------------------------------------------
# 🧭 Appel de la fonction sur ton jeu de modèles OLS
# ----------------------------------------------------------
perm_df = permutation_importance_pseudo_oos(
    models=models,
    df_train_global=df_train_global,
    cols_tx=cols_tx,
    h=h,
    n_repeats=20  # augmente à 50 pour des résultats plus stables
)

print("\n=== 🔍 Importance par permutation (pseudo–OOS) ===")
print(perm_df.head(15).to_string(index=False))


=== 🔍 Importance par permutation (pseudo–OOS) ===
       variable  perm_mae_ratio_mean  perm_mae_ratio_std  n_windows
          USREC             1.253742            0.135382         27
          TB3MS             1.034833            0.029633         27
        S&P 500             1.012959            0.004565         27
DPCERA3M086SBEA             1.000301            0.000418         27
         INDPRO             1.000260            0.001776         27
      OILPRICEx             1.000161            0.000390         27
            RPI             1.000154            0.000394         27
       BUSLOANS             1.000140            0.000181         27
           M2SL             1.000133            0.000219         27
       CPIAUCSL             1.000030            0.000138         27


👉 Règle simple :
- Ratio ≈ 1 → la variable n’apporte pratiquement rien.
- Ratio > 1.05 → variable réellement utile (la perte d’information est notable).
- Ratio > 1.20 → forte importance.

Ton modèle OLS prédit principalement le chômage via le cycle économique :
- USREC (récession ou non) est la variable clé — sa permutation dégrade la performance de ~26 %.
- TB3MS (taux court) les taux d’intérêt à court terme apportent une petite amélioration (signal de politique monétaire).
- Les autres variables ont un effet négligeable.

Conclusion : À horizon 12 mois, le modèle OLS base sa prévision du chômage principalement sur la présence ou non de récession.
Les indicateurs financiers jouent un rôle secondaire, tandis que les autres variables macro (activité, prix, monnaie) ont une contribution négligeable dans les performances pseudo–OOS.

In [43]:
# ==========================================================
# 💡 IMPORTANCE SHAPLEY (pour modèle OLS)
# ==========================================================
import shap
import numpy as np
import pandas as pd

# 1️⃣ On utilise le dernier modèle entraîné
model_final = models[-1]

# 2️⃣ On reconstitue ses données finales (dernière fenêtre du train)
df_final = df_train_global.copy()
X_full = df_final[cols_tx].iloc[:-h].copy()
Y_full = df_final["UNRATE"].shift(-h).iloc[:-h].copy()
valid = ~(X_full.isnull().any(axis=1) | Y_full.isnull())
X_full = X_full.loc[valid]
Y_full = Y_full.loc[valid]

# 3️⃣ Calcul des valeurs SHAP
# Pour les modèles linéaires, on peut utiliser shap.LinearExplainer (plus stable)
explainer = shap.LinearExplainer(model_final, X_full, feature_perturbation="interventional")
shap_values = explainer(X_full)

# 4️⃣ Importance moyenne absolue
shap_df = pd.DataFrame({
    "variable": X_full.columns,
    "shap_mean_abs": np.abs(shap_values.values).mean(axis=0),
})
shap_df["shap_share"] = shap_df["shap_mean_abs"] / shap_df["shap_mean_abs"].sum()
shap_df = shap_df.sort_values("shap_mean_abs", ascending=False).reset_index(drop=True)

# 5️⃣ Affichage
print("\n=== 💡 Importance SHAP (shares) ===")
print(shap_df.head(10).to_string(index=False))


=== 💡 Importance SHAP (shares) ===
       variable  shap_mean_abs  shap_share
          USREC       0.342433    0.644562
          TB3MS       0.167207    0.314733
        S&P 500       0.017503    0.032945
      OILPRICEx       0.001360    0.002560
DPCERA3M086SBEA       0.001169    0.002200
         INDPRO       0.000662    0.001246
            RPI       0.000381    0.000717
       BUSLOANS       0.000257    0.000484
           M2SL       0.000188    0.000353
       CPIAUCSL       0.000105    0.000198


c:\Users\Mita\AppData\Local\Programs\Python\Python312\Lib\site-packages\shap\explainers\_linear.py:99: FutureWarning: The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, maskers.Partition or maskers.Impute).
  warnings.warn(wmsg, FutureWarning)


- USREC (0.64)	💥 Variable dominante (64 %) — la moitié des variations prédites du chômage viennent du seul indicateur de récession. Quand USREC = 1, le modèle anticipe une hausse significative du chômage.	

- TB3MS (0.31)	⚙️ Deuxième facteur clé (31 %) — les taux courts jouent un rôle important : ils influencent fortement les prévisions à 12 mois, probablement comme proxy de la politique monétaire restrictive.

Les valeurs SHAP confirment que la récession (USREC) et les taux d’intérêt à court terme (TB3MS) sont les deux principaux moteurs des prévisions du modèle linéaire.
Ensemble, ils expliquent près de 95 % de la contribution totale aux variations prédites du chômage.

Les autres variables macroéconomiques ont un poids négligeable, ce qui souligne la simplicité et la nature cyclique du lien entre conjoncture et chômage.

# 📊 Métriques utilisées

## 1) Performance globale

**Coefficient de détermination (R²)**  
$$
R^2 = 1 - \frac{\sum_i (y_i - \hat{y}_i)^2}{\sum_i (y_i - \bar{y})^2}
$$  
➡️ Part de la variance expliquée par le modèle (0 = pas mieux que la moyenne, 1 = parfait).

**Erreur absolue moyenne (MAE)**  
$$
MAE = \frac{1}{n}\sum_{i=1}^n |y_i - \hat{y}_i|
$$  
➡️ Écart absolu moyen entre valeurs réelles et prédites, robuste aux outliers.

**Erreur quadratique moyenne (RMSE)**  
$$
RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2}
$$  
➡️ Similaire au MAE mais pénalise davantage les grosses erreurs.

**Corrélation de Pearson**  
$$
\rho(y, \hat{y}) = \frac{\text{Cov}(y, \hat{y})}{\sigma_y \cdot \sigma_{\hat{y}}}
$$  
➡️ Mesure le degré de lien linéaire entre les prédictions et les observations.

**Abs Mean Deviance (AMD)**  
$$
AMD = \frac{1}{n}\sum_{i=1}^n |\hat{y}_i - \bar{\hat{y}}|
$$  
➡️ Écart moyen des prédictions par rapport à leur moyenne ; sert de référence pour la permutation prédiction-basée.

---

## 2) Importance par permutation
La relation entre Y et X dépend du temps. Quand on perturbe la série X (en la mélangeant), on casse ce lien, et si l’erreur augmente, cela montre que X est une variable clé pour expliquer Y.

**Ratio MAE**  
$$
PI^{MAE}_j = \frac{MAE^{(perm)}_j}{MAE^{(base)}}
$$  
➡️ Si > 1, la variable est utile pour réduire l’erreur absolue.

**Ratio RMSE**  
$$
PI^{RMSE}_j = \frac{RMSE^{(perm)}_j}{RMSE^{(base)}}
$$  
➡️ Si > 1, la variable aide à limiter les grosses erreurs.

**Déviance de prédiction**  
$$
PI^{dev}_j = \frac{1}{n}\sum_{i=1}^n \big|\hat{y}_i - \hat{y}^{(perm)}_{i,j}\big|
$$  
➡️ Mesure combien les prédictions changent quand on brouille une variable.

---

## 3) Importance Shapley

**Décomposition des prédictions**  
$$
\hat{y}_i = \phi_0 + \sum_{j=1}^p \phi_{ij}
$$  
➡️ Chaque prédiction est expliquée par une contribution \(\phi_{ij}\) par variable.

**Importance absolue moyenne**  
$$
\text{Mean}(|\phi_j|) = \frac{1}{n}\sum_{i=1}^n |\phi_{ij}|
$$  
➡️ Contribution moyenne (absolue) d’une variable sur toutes les prédictions.

**Shapley share**  
$$
\Gamma_j = \frac{\text{Mean}(|\phi_j|)}{\sum_{k=1}^p \text{Mean}(|\phi_k|)}
$$  
➡️ Part relative de la variable dans l’explication totale (somme des parts = 1).

## Interprétation des résultats 

### Performance globale 
- R² = 0.2266 → modèle OLS explique ~23 % de la variance du chômage US.
- MAE = 0.6774 → en moyenne, l’erreur absolue est de 0.68 points (dans l’unité de la variable cible).
- RMSE = 0.8750 → un peu plus élevé que le MAE, ce qui indique la présence de grosses erreurs ponctuelles.
- Corrélation = 0.4760 (p ≈ 10⁻³⁵) → lien positif et significatif entre prédictions et observations, mais seulement modéré. Ce qui peut expliquer la présence d'une relation 
- Abs Mean Deviance = 0.3370 → sert ici de référence pour l’importance prédiction-basée : les prédictions s’écartent en moyenne de 0.34 de leur propre moyenne.

Lecture : le modèle OLS capte une partie utile du signal, mais laisse beaucoup de variance inexpliquée. La corrélation faible illustre éventuellement la présence des relations non-linéaire, et non captées par OLS.

### 🔹 2. Importance par permutation
- INDPRO (Industrial Production) : la plus influente. Sa permutation augmente MAE de +19 % et RMSE de +20 %, avec une forte déviance de prédiction (0.41).
- TB3MS (Taux d’intérêt à 3 mois) : impact non négligeable, ratios ~1.02 et déviance ~0.10.
- BUSLOANS (Prêts commerciaux) : rôle similaire (MAE ratio 1.017, déviance ~0.09).
- S&P 500 : contribution modérée, ratios légèrement > 1.
- RPI, M2SL : influence plus faible mais perceptible.
- CPIAUCSL, OILPRICEx, DPCERA3M086SBEA : quasi neutres (ratios ≈ 1, déviance très faible).

Lecture : INDPRO domine largement la performance, les autres apportent des compléments mais plus modestes.

### 🔹 3. Importance Shapley (shares)
- INDPRO : ~52 % de l’explication totale des prédictions → cohérence parfaite avec la permutation.
- TB3MS (12 %) + BUSLOANS (12 %) : deux autres piliers importants.
- S&P 500 (9,7 %) : contribue de façon notable.
- M2SL (5 %), RPI (3 %), CPIAUCSL (3,5 %) : apports plus secondaires.
- OILPRICEx et DPCERA3M086SBEA (<2 %) : quasi négligeables dans ce modèle.

Lecture : INDPRO est la variable macroéconomique centrale, suivie par des indicateurs financiers (taux courts, prêts bancaires, marché actions).

# Test

In [ ]:
import joblib

saved_model = joblib.load("models/model_final.pkl")
model_final = saved_model["model"]
features = saved_model["features"]
winsor_level = saved_model["winsor_level"]
norm_var = saved_model["norm_var"]
mean_full = saved_model["mean_full"]
std_full = saved_model["std_full"]